<a href="https://colab.research.google.com/github/sohomghosh/FinCausal-2020/blob/master/RE_TRAIN_FinCausal_2020_best_solution_score_2022data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Always run at the begining

In [ ]:
!rm -rf /content/FinCausal-2020/
!git clone https://github.com/sohomghosh/FinCausal-2020.git

Cloning into 'FinCausal-2020'...
remote: Enumerating objects: 316, done.
remote: Counting objects: 100% (316/316), done.
remote: Compressing objects: 100% (240/240), done.
remote: Total 316 (delta 92), reused 286 (delta 73), pack-reused 0
Receiving objects: 100% (316/316), 1.55 MiB | 7.05 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [ ]:
!pip install -r /content/FinCausal-2020/requirements.txt

In [ ]:
%%capture
!pip install tensorflow

In [ ]:
# Restart run-time

In [ ]:
import stanza
stanza.download('en')

2022-04-16 12:54:33 INFO: Downloading default packages for language: en (English)...
2022-04-16 12:55:57 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
from google.colab import files

In [ ]:
import os
os.chdir('/content/FinCausal-2020/')

# [DONE] One time Only for training and test data pre-processing

In [ ]:
# Renamed data/pos.txt to data/pos_tags.txt
! mv /content/FinCausal-2020/data/pos.txt /content/FinCausal-2020/data/pos_tags.txt

In [ ]:
!wget https://raw.githubusercontent.com/sohomghosh/CEPN/main/FINCAUSAL_data_all_till2022_dedup_clean_formatted.csv -P /content/FinCausal-2020/data

--2022-04-16 09:56:56--  https://raw.githubusercontent.com/sohomghosh/CEPN/main/FINCAUSAL_data_all_till2022_dedup_clean_formatted.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1395026 (1.3M) [text/plain]
Saving to: ‘/content/FinCausal-2020/data/FINCAUSAL_data_all_till2022_dedup_clean_formatted.csv’

FINCAUSAL_data_all_ 100%[===================>]   1.33M  --.-KB/s    in 0.05s   

2022-04-16 09:56:56 (25.4 MB/s) - ‘/content/FinCausal-2020/data/FINCAUSAL_data_all_till2022_dedup_clean_formatted.csv’ saved [1395026/1395026]



In [ ]:
import pandas as pd
train_data = pd.read_csv('/content/FinCausal-2020/data/FINCAUSAL_data_all_till2022_dedup_clean_formatted.csv', sep = ';', header = None)
train_data.columns = ['index','text',	'cause',	'effect',	'cause_start',	'effect_start']
train_data.to_csv('/content/FinCausal-2020/data/train.csv', sep = ';', index = False)
#This will overwrite the previous train.csv

In [ ]:
!head -2 /content/FinCausal-2020/data/train.csv

index;text;cause;effect;cause_start;effect_start
0;The increase in net interest income was due primarily to a $152.9 billion increase in average outstanding loans, a $32.6 billion increase in average securities, partially offset by a 78 basis point decrease in earning asset yields.NIM was 3.22% for 2020, down 20 basis points compared to the prior year.;a $152.9 billion increase in average outstanding loans, a $32.6 billion increase in average securities, partially offset by a 78 basis point decrease in earning asset yields.;The increase in net interest income;57;0


In [ ]:
!python /content/FinCausal-2020/preprocess.py /content/FinCausal-2020/data/train.csv train

transformation example:  {'sentence': 'Additional increases in noninterest income were primarily due to higher insurance income driven by improved production levels and acquisitions.', 'cause': 'higher insurance income driven by improved production levels and acquisitions.', 'effect': 'Additional increases in noninterest income'}
100% 2808/2808 [08:00<00:00,  5.84it/s]


In [ ]:
#train.txt has been generated for all data till 2022

In [ ]:
#blind text has been created from blind_fnp2022-fincausal-task2.csv by using proper formatting and encoding
!python /content/FinCausal-2020/preprocess.py /content/FinCausal-2020/data/blind_test_2022.csv test

transformation example:  {'sentence': ' Instead, the company reported itself as a family business, which is liable to a single 32% tax.'}
100% 933/933 [02:20<00:00,  6.66it/s]


In [ ]:
#test.txt has been generated for 2022

# For Predictions only from pre-trained model

In [ ]:
# download and upload the file separately from https://drive.google.com/file/d/1omc-hy4uAb1JaeVrNQvbvGOQ3tZga7C3/view
!unzip /content/FinCausal-2020/eval_bio.zip -d /content/FinCausal-2020/

Archive:  /content/FinCausal-2020/eval_bio.zip
   creating: /content/FinCausal-2020/eval_bio/
  inflating: /content/FinCausal-2020/eval_bio/test_predictions.txt  
  inflating: /content/FinCausal-2020/eval_bio/.DS_Store  
  inflating: /content/FinCausal-2020/__MACOSX/eval_bio/._.DS_Store  
  inflating: /content/FinCausal-2020/eval_bio/tokenizer_config.json  
  inflating: /content/FinCausal-2020/eval_bio/special_tokens_map.json  
  inflating: /content/FinCausal-2020/eval_bio/config.json  
  inflating: /content/FinCausal-2020/eval_bio/training_args.bin  
  inflating: /content/FinCausal-2020/eval_bio/vocab.txt  
  inflating: /content/FinCausal-2020/eval_bio/pytorch_model.bin  


# Train & Predictions

In [ ]:
!mkdir /content/FinCausal-2020/pred/

## Training BERT

In [ ]:
%%bash
export MAX_LENGTH=350
export BERT_MODEL="bert-base-uncased"

export DATA_DIR=data
export OUTPUT_DIR=pred
export SEED=1

CUDA_VISIBLE_DEVICES=0 python run.py \
--data_dir $DATA_DIR \
--labels $DATA_DIR/labels.txt \
--model_name_or_path $BERT_MODEL \
--output_dir $OUTPUT_DIR \
--max_seq_length  $MAX_LENGTH \
--seed $SEED \
--do_train \

get_labels: ['_', 'B-C', 'I-C', 'B-E', 'I-E']
pos_labels: ['<pad>', '$', "''", ',', '-LRB-', '-RRB-', '.', ':', 'ADD', 'AFX', 'CC', 'CD', 'DT', 'EX', 'FW', 'GW', 'HYPH', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NFP', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']
Number of pos_labels: 50
ADD_POS: False
pos_labels: ['<pad>', '$', "''", ',', '-LRB-', '-RRB-', '.', ':', 'ADD', 'AFX', 'CC', 'CD', 'DT', 'EX', 'FW', 'GW', 'HYPH', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NFP', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']
Number of pos_labels: 50
{"learning_rate": 2.6258309591642926e-05, "loss": 0.553298275668174, "step": 500}
{"learning_rate": 2.51661918328585e-06, "loss": 0.2587094704005867, "step": 1000}


04/16/2022 10:23:34 - INFO - transformers.training_args -   PyTorch: setting up devices
04/16/2022 10:23:34 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/16/2022 10:23:34 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='pred', overwrite_output_dir=False, add_pos=False, do_train=True, do_eval=False, do_predict=False, evaluate_during_training=False, per_gpu_train_batch_size=8, per_gpu_eval_batch_size=8, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir=None, logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=1, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=False)
04/16/2022 10:23:34 - INFO - filelock -   Lock 139704458777232 acquired on /root/.cache/torch/transformer

In [ ]:
!zip -r /content/FinCausal-2020/model_bert_till_2022_train.zip /content/FinCausal-2020/pred/*.*

  adding: content/FinCausal-2020/pred/config.json (deflated 61%)
  adding: content/FinCausal-2020/pred/pytorch_model.bin (deflated 7%)
  adding: content/FinCausal-2020/pred/special_tokens_map.json (deflated 40%)
  adding: content/FinCausal-2020/pred/tokenizer_config.json (stored 0%)
  adding: content/FinCausal-2020/pred/training_args.bin (deflated 43%)
  adding: content/FinCausal-2020/pred/vocab.txt (deflated 53%)


In [ ]:
files.download("/content/FinCausal-2020/model_bert_till_2022_train.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Predictions BERT

In [ ]:
!mkdir /content/FinCausal-2020/pred_new_model/

In [ ]:
%%bash
export MAX_LENGTH=350
export BERT_MODEL="/content/FinCausal-2020/pred/"

export DATA_DIR=data
export OUTPUT_DIR=pred_new_model
export SEED=1

CUDA_VISIBLE_DEVICES=0 python run.py \
--data_dir $DATA_DIR \
--labels $DATA_DIR/labels.txt \
--model_name_or_path $BERT_MODEL \
--output_dir $OUTPUT_DIR \
--max_seq_length  $MAX_LENGTH \
--seed $SEED \
--do_predict \

get_labels: ['_', 'B-C', 'I-C', 'B-E', 'I-E']
pos_labels: ['<pad>', '$', "''", ',', '-LRB-', '-RRB-', '.', ':', 'ADD', 'AFX', 'CC', 'CD', 'DT', 'EX', 'FW', 'GW', 'HYPH', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NFP', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']
Number of pos_labels: 50
ADD_POS: False
pos_labels: ['<pad>', '$', "''", ',', '-LRB-', '-RRB-', '.', ':', 'ADD', 'AFX', 'CC', 'CD', 'DT', 'EX', 'FW', 'GW', 'HYPH', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NFP', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']
Number of pos_labels: 50
Number of exact matches: 2 out of 933
933


04/16/2022 10:50:26 - INFO - transformers.training_args -   PyTorch: setting up devices
04/16/2022 10:50:26 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/16/2022 10:50:26 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='pred_new_model', overwrite_output_dir=False, add_pos=False, do_train=False, do_eval=False, do_predict=True, evaluate_during_training=False, per_gpu_train_batch_size=8, per_gpu_eval_batch_size=8, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir=None, logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=1, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=False)
04/16/2022 10:50:26 - INFO - transformers.configuration_utils -   loading configuration file /c

In [ ]:
!python submission.py pred_new_model/test_predictions.txt data/blind_test_2022.csv bert_till2022_trained_output_test.csv

-----------------------------------------------
pred_new_model/test_predictions.txt : 933
['B-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', '_', 'B-E', 'I-E', 'I-E', 'I-E', 'I-E', 'I-E', 'I-E', 'I-E', 'I-E', 'I-E']
Length of eval data: 933


In [ ]:
#bert_till2022_trained_output_test.csv is the file for submission

In [ ]:
!rm -rf pred

## Train bert-base-uncased without POS

In [ ]:
!mkdir /content/FinCausal-2020/bert_base_no_pos/

In [ ]:
%%bash
export MAX_LENGTH=350
export BERT_MODEL="bert-base-uncased"

export DATA_DIR=data
export OUTPUT_DIR=bert_base_no_pos
export SEED=1

CUDA_VISIBLE_DEVICES=0 python run_bert_no_pos.py \
--data_dir $DATA_DIR \
--labels $DATA_DIR/labels.txt \
--model_name_or_path $BERT_MODEL \
--output_dir $OUTPUT_DIR \
--max_seq_length  $MAX_LENGTH \
--seed $SEED \
--do_train \




Process is terminated.


## Predictions BERT without pos

In [ ]:
!mkdir /content/FinCausal-2020/pred_new_bert_model_no_pos/

In [ ]:
%%bash
export MAX_LENGTH=350
export BERT_MODEL="/content/FinCausal-2020/bert_base_no_pos/"

export DATA_DIR=data
export OUTPUT_DIR=pred_new_bert_model_no_pos
export SEED=1

CUDA_VISIBLE_DEVICES=0 python run_bert_no_pos.py \
--data_dir $DATA_DIR \
--labels $DATA_DIR/labels.txt \
--model_name_or_path $BERT_MODEL \
--output_dir $OUTPUT_DIR \
--max_seq_length  $MAX_LENGTH \
--seed $SEED \
--do_predict \

get_labels: ['_', 'B-C', 'I-C', 'B-E', 'I-E']
ADD_POS: False
Number of exact matches: 2 out of 933
933


04/16/2022 12:28:44 - INFO - transformers.training_args -   PyTorch: setting up devices
04/16/2022 12:28:44 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/16/2022 12:28:44 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='pred_new_bert_model_no_pos', overwrite_output_dir=False, add_pos=False, do_train=False, do_eval=False, do_predict=True, evaluate_during_training=False, per_gpu_train_batch_size=8, per_gpu_eval_batch_size=8, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir=None, logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=1, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=False)
04/16/2022 12:28:44 - INFO - transformers.configuration_utils -   loading configura

In [ ]:
!python submission.py pred_new_bert_model_no_pos/test_predictions.txt data/blind_test_2022.csv bert_till2022_trained_output_test_no_pos.csv

python3: can't open file 'submission.py': [Errno 2] No such file or directory


## RoBERTa Large Train

In [ ]:
!mkdir /content/FinCausal-2020/roberta_large/

mkdir: cannot create directory ‘/content/FinCausal-2020/roberta_large/’: File exists


In [ ]:
!rm /content/FinCausal-2020/run_roberta.py
!rm /content/FinCausal-2020/nonpos_utils.py
!wget https://raw.githubusercontent.com/sohomghosh/FinCausal-2020/master/run_roberta.py -P /content/FinCausal-2020/
!wget https://raw.githubusercontent.com/sohomghosh/FinCausal-2020/master/nonpos_utils.py -P /content/FinCausal-2020/

--2022-04-16 13:51:39--  https://raw.githubusercontent.com/sohomghosh/FinCausal-2020/master/run_roberta.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12387 (12K) [text/plain]
Saving to: ‘/content/FinCausal-2020/run_roberta.py’

run_roberta.py      100%[===================>]  12.10K  --.-KB/s    in 0s      

2022-04-16 13:51:39 (103 MB/s) - ‘/content/FinCausal-2020/run_roberta.py’ saved [12387/12387]

--2022-04-16 13:51:39--  https://raw.githubusercontent.com/sohomghosh/FinCausal-2020/master/nonpos_utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaitin

In [ ]:
%%bash
export MAX_LENGTH=350
export BERT_MODEL="roberta-base"

export DATA_DIR=data
export OUTPUT_DIR=roberta_large
export SEED=1

CUDA_VISIBLE_DEVICES=0 python run_roberta.py \
--data_dir $DATA_DIR \
--labels $DATA_DIR/labels.txt \
--model_name_or_path $BERT_MODEL \
--output_dir $OUTPUT_DIR \
--max_seq_length  $MAX_LENGTH \
--seed $SEED \
--do_train \

get_labels: ['_', 'B-C', 'I-C', 'B-E', 'I-E']


04/16/2022 13:58:13 - INFO - transformers.training_args -   PyTorch: setting up devices
04/16/2022 13:58:13 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
04/16/2022 13:58:13 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='roberta_large', overwrite_output_dir=False, add_pos=False, do_train=True, do_eval=False, do_predict=False, evaluate_during_training=False, per_gpu_train_batch_size=8, per_gpu_eval_batch_size=8, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir=None, logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=1, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=False)
04/16/2022 13:58:13 - INFO - transformers.configuration_utils -   loading configuration file http

## Predictions RoBERTa large no pos

In [ ]:
!mkdir /content/FinCausal-2020/pred_roberta_large_no_pos/

In [ ]:
%%bash
export MAX_LENGTH=350
export BERT_MODEL="/content/FinCausal-2020/roberta_large/"

export DATA_DIR=data
export OUTPUT_DIR=pred_roberta_large_no_pos
export SEED=1

CUDA_VISIBLE_DEVICES=0 python run_roberta.py \
--data_dir $DATA_DIR \
--labels $DATA_DIR/labels.txt \
--model_name_or_path $BERT_MODEL \
--output_dir $OUTPUT_DIR \
--max_seq_length  $MAX_LENGTH \
--seed $SEED \
--do_predict \

In [ ]:
!python submission.py pred_roberta_large_no_pos/test_predictions.txt data/blind_test_2022.csv pred_roberta_large_no_pos_test_output.csv

## Domain specific train - BERT-SEC no-pos

In [ ]:
!mkdir /content/FinCausal-2020/bert-sec/

In [ ]:
!git clone https://huggingface.co/nlpaueb/sec-bert-base

Cloning into 'sec-bert-base'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 121 (delta 8), reused 0 (delta 0), pack-reused 106
Receiving objects: 100% (121/121), 123.94 KiB | 2.10 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [ ]:
%%bash
export MAX_LENGTH=350
export BERT_MODEL="sec-bert-base"

export DATA_DIR=data
export OUTPUT_DIR=bert-sec
export SEED=1

CUDA_VISIBLE_DEVICES=0 python run_domain_specific.py \
--data_dir $DATA_DIR \
--labels $DATA_DIR/labels.txt \
--model_name_or_path $BERT_MODEL \
--output_dir $OUTPUT_DIR \
--max_seq_length  $MAX_LENGTH \
--seed $SEED \
--do_train \




Process is terminated.


## Predictions Domain specific train - BERT-SEC no-pos

In [ ]:
!mkdir /content/FinCausal-2020/pred_bert_sec_no_pos/

In [ ]:
%%bash
export MAX_LENGTH=350
export BERT_MODEL="/content/FinCausal-2020/bert-sec/"

export DATA_DIR=data
export OUTPUT_DIR=pred_bert_sec_no_pos
export SEED=1

CUDA_VISIBLE_DEVICES=0 python run_domain_specific.py \
--data_dir $DATA_DIR \
--labels $DATA_DIR/labels.txt \
--model_name_or_path $BERT_MODEL \
--output_dir $OUTPUT_DIR \
--max_seq_length  $MAX_LENGTH \
--seed $SEED \
--do_predict \

In [ ]:
!python submission.py pred_bert_sec_no_pos/test_predictions.txt data/blind_test_2022.csv pred_bert_sec_no_pos_test_output.csv

## Domain specific train - BERT-SEC no-pos

In [ ]:
!mkdir /content/FinCausal-2020/finbert-cau/

In [ ]:
!git clone https://huggingface.co/ProsusAI/finbert

Cloning into 'finbert'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 49 (delta 23), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [ ]:
%%bash
export MAX_LENGTH=350
export BERT_MODEL="finbert"

export DATA_DIR=data
export OUTPUT_DIR=finbert-cau
export SEED=1

CUDA_VISIBLE_DEVICES=0 python run_domain_specific.py \
--data_dir $DATA_DIR \
--labels $DATA_DIR/labels.txt \
--model_name_or_path $BERT_MODEL \
--output_dir $OUTPUT_DIR \
--max_seq_length  $MAX_LENGTH \
--seed $SEED \
--do_train \


Process is terminated.


## Predictions Domain specific train - FinBERT no-pos

In [ ]:
!mkdir /content/FinCausal-2020/pred_finbert_no_pos/

In [ ]:
%%bash
export MAX_LENGTH=350
export BERT_MODEL="/content/FinCausal-2020/finbert-cau/"

export DATA_DIR=data
export OUTPUT_DIR=pred_finbert_no_pos
export SEED=1

CUDA_VISIBLE_DEVICES=0 python run_domain_specific.py \
--data_dir $DATA_DIR \
--labels $DATA_DIR/labels.txt \
--model_name_or_path $BERT_MODEL \
--output_dir $OUTPUT_DIR \
--max_seq_length  $MAX_LENGTH \
--seed $SEED \
--do_predict \

In [ ]:
!python submission.py pred_finbert_no_pos/test_predictions.txt data/blind_test_2022.csv pred_finbert_no_pos_test_output.csv

## Retrain previously trained Fincausal Model

In [ ]:
# download and upload the file separately from https://drive.google.com/file/d/1omc-hy4uAb1JaeVrNQvbvGOQ3tZga7C3/view
!unzip /content/FinCausal-2020/eval_bio.zip -d /content/FinCausal-2020/

In [ ]:
%%bash
export MAX_LENGTH=350
export BERT_MODEL="eval_bio"

export DATA_DIR=data
export OUTPUT_DIR=fincau2020_retrain
export SEED=1

CUDA_VISIBLE_DEVICES=0 python run_domain_specific.py \
--data_dir $DATA_DIR \
--labels $DATA_DIR/labels.txt \
--model_name_or_path $BERT_MODEL \
--output_dir $OUTPUT_DIR \
--max_seq_length  $MAX_LENGTH \
--seed $SEED \
--do_train \


## Predictions Domain specific train - FinCausal-2020 extend no-pos

In [ ]:
!mkdir /content/FinCausal-2020/pred_fcausal2020_ex_no_pos/

In [ ]:
%%bash
export MAX_LENGTH=350
export BERT_MODEL="/content/FinCausal-2020/fincau2020_retrain/"

export DATA_DIR=data
export OUTPUT_DIR=pred_fcausal2020_ex_no_pos
export SEED=1

CUDA_VISIBLE_DEVICES=0 python run_domain_specific.py \
--data_dir $DATA_DIR \
--labels $DATA_DIR/labels.txt \
--model_name_or_path $BERT_MODEL \
--output_dir $OUTPUT_DIR \
--max_seq_length  $MAX_LENGTH \
--seed $SEED \
--do_predict \

In [ ]:
!python submission.py pred_fcausal2020_ex_no_pos/test_predictions.txt data/blind_test_2022.csv pred_fcausal2020_ex_no_pos_test_output.csv